In [1]:
#import modules
import numpy as np
import pandas as pd
import time
from keras import backend as K 
from keras.engine.training import Model
from keras.utils import to_categorical, plot_model
from keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dropout, Activation, Flatten
from keras.layers import LSTM, Dense, Input
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error

In [2]:
#import data
df = pd.read_csv('../Sub0-RAW.csv')
df.head()

,Dates,Year,Month,Day,Hours,Precipitation,Temperature,Outflow
0,10/1/1980,1980,10,1,0:00:00,0.0,102.992,5.0
1,10/1/1980,1980,10,1,1:00:00,0.0,97.790,5.0
2,10/1/1980,1980,10,1,2:00:00,0.0,92.588,5.0
3,10/1/1980,1980,10,1,3:00:00,0.0,87.404,5.0
4,10/1/1980,1980,10,1,4:00:00,0.0,85.172,5.0


In [3]:
#define train and label column
train_cols = ["Outflow"]
label_cols = ["Outflow(t+1)"]

In [4]:
#filter the columns
new_df = df.filter(train_cols, axis=1)

In [5]:
#Min Max scalr normalizing
xtrain_min_max_scaler = MinMaxScaler(feature_range = (0, 1))
ytrain_min_max_scaler = MinMaxScaler(feature_range = (0, 1))
xtest_min_max_scaler = MinMaxScaler(feature_range = (0, 1))
ytest_min_max_scaler = MinMaxScaler(feature_range = (0, 1))

In [6]:
#function to shift the time_series data for getting labels
def lag_seq(df, n_seq):
    for i in range(n_seq):
        df['Outflow(t+%d)' %(i+1)] = new_df['Outflow'].shift(-(i+1))
    return df

# building timeseries data with given timesteps
def timeseries(X, Y, Y_actual, time_steps, out_steps):
    input_size_0 = X.shape[0] - time_steps
    input_size_1 = X.shape[1]
    X_values = np.zeros((input_size_0, time_steps, input_size_1))
    Y_values = np.zeros((input_size_0,))
    Y_values_actual = np.zeros((input_size_0,))
    
    for i in tqdm_notebook(range(input_size_0)):
        X_values[i] = X[i:time_steps+i]
        Y_values[i] = Y[time_steps+i-1, 0]
        Y_values_actual[i] = Y_actual[time_steps+i-1, 0]
        
    print("length of time-series i/o",X_values.shape,Y_values.shape)
    return X_values, Y_values, Y_values_actual

#getting data ready for training the model
def data_processing(lag_df):
    #Splitting training and test data
    df_train, df_test = train_test_split(lag_df, train_size=0.8, test_size=0.2, shuffle=False)
    x_train = df_train.loc[:,train_cols].values
    y_train = df_train.loc[:,label_cols].values
    x_test = df_test.loc[:,train_cols].values
    y_test = df_test.loc[:,label_cols].values    
   
    #Normalizing training data
    x_train_nor = xtrain_min_max_scaler.fit_transform(x_train)
    y_train_nor = ytrain_min_max_scaler.fit_transform(y_train)

    # Normalizing test data
    x_test_nor = xtest_min_max_scaler.fit_transform(x_test)
    y_test_nor = ytest_min_max_scaler.fit_transform(y_test)
    
    # Saving actual train and test y_label
    y_train_actual = y_train
    y_test_actual = y_test
    
    return x_train_nor, y_train_nor, y_train_actual, x_test_nor, y_test_nor, y_test_actual

In [7]:
#calling function to create lag dataframe
lag_df = lag_seq(new_df, 1)
lag_df.dropna(inplace=True)

In [8]:
#Building timeseries
x_train_nor, y_train_nor, y_train_actual, x_test_nor, y_test_nor, y_test_actual = data_processing(lag_df)
X_Train, Y_Train, Y_train_actual = timeseries(x_train_nor, y_train_nor, y_train_actual, time_steps=18, out_steps=1)
X_Test, Y_Test, Y_test_actual = timeseries(x_test_nor, y_test_nor, y_test_actual, time_steps=18, out_steps=1)

/home/bidur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



length of time-series i/o (148416, 18, 1) (148416,)



length of time-series i/o (37091, 18, 1) (37091,)


In [9]:
#Build Model
def make_model(X_Train):
    input_layer = Input(shape=(X_Train.shape[1], X_Train.shape[2]))

    lstm1 = LSTM(units=16, return_sequences=True)(input_layer)
    dropout1 = Dropout(0.2)(lstm1)
    lstm2 = LSTM(units=32, return_sequences=True)(dropout1)
    dropout2 = Dropout(0.2)(lstm2)
    lstm3 = LSTM(units=64)(dropout2)
    dropout3 = Dropout(0.2)(lstm3)

    dense1 = Dense(128, activation='relu')(dropout3)
    dense2 = Dense(64, activation='relu')(dense1)
    output_layer = Dense(1, activation='linear')(dense2)
    
    return Model(inputs=input_layer, outputs=output_layer)


In [10]:
mse_for_iter = []
train_loss_over_epoch = []
val_loss_over_epoch = []

def run_iteration(no_iter=20, epochs = 10):
    start_time = time.time()
    for iteration in range(no_iter):
        print(f'Iteration: {iteration + 1}')
        K.clear_session()
        model = make_model(X_Train)
        #plot_model(model, show_shapes=True)
        model.compile(optimizer = 'adam', loss = 'mean_squared_error')
        history = model.fit(X_Train, Y_Train, epochs=epochs,
                       validation_data=(X_Test, Y_Test))
        train_loss_over_epoch.append(history.history['loss'])
        val_loss_over_epoch.append(history.history['val_loss'])
        predicted_y_values = model.predict(X_Test)
        predicted_y_values_unnor = ytrain_min_max_scaler.inverse_transform(predicted_y_values)
        mse_for_iter.append(mean_squared_error(predicted_y_values_unnor, Y_test_actual))
               
    end_time = time.time()
    print('--------------------------\n')
    print(f'Starting time: {start_time}')
    print(f'Completing time: {end_time}')
    print(f'It took {(start_time - end_time)/60} minutes to train the model for {no_iter} iterations')
    #return predicted unnormalized values for test set from last iteration 
    return predicted_y_values, predicted_y_values_unnor, model

predicted_y_values, predicted_y_values_unnor, model = run_iteration(1,10)

Iteration: 1
Epoch 1/10
4638/4638 [==============================] - 95s 20ms/step - loss: 1.2994e-04 - val_loss: 7.1362e-05
Epoch 2/10
4638/4638 [==============================] - 93s 20ms/step - loss: 5.5565e-05 - val_loss: 4.1709e-05
Epoch 3/10
4638/4638 [==============================] - 93s 20ms/step - loss: 3.9301e-05 - val_loss: 1.6506e-05
Epoch 4/10
4638/4638 [==============================] - 93s 20ms/step - loss: 3.7763e-05 - val_loss: 3.2022e-05
Epoch 5/10
4638/4638 [==============================] - 93s 20ms/step - loss: 2.9755e-05 - val_loss: 1.2832e-05
Epoch 6/10
4638/4638 [==============================] - 92s 20ms/step - loss: 2.6578e-05 - val_loss: 3.1200e-05
Epoch 7/10
4638/4638 [==============================] - 93s 20ms/step - loss: 2.2002e-05 - val_loss: 1.2047e-05
Epoch 8/10
4638/4638 [==============================] - 93s 20ms/step - loss: 2.1778e-05 - val_loss: 1.3413e-05
Epoch 9/10
4638/4638 [==============================] - 93s 20ms/step - loss: 1.9775e-05 - 

In [11]:
#prediction for X_Train values which has to be used to predict another day output
y_train_predict = model.predict(X_Train)

In [12]:
#for model2
#Building timeseries
X_Train1, Y_Train1, Y_train_actual1 = timeseries(x_train_nor, y_train_nor, y_train_actual, time_steps=19, out_steps=1)
X_Test1, Y_Test1, Y_test_actual1 = timeseries(x_test_nor, y_test_nor, y_test_actual, time_steps=19, out_steps=1)

/home/bidur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app



length of time-series i/o (148415, 19, 1) (148415,)



length of time-series i/o (37090, 19, 1) (37090,)


In [13]:
#replacing last value of each timeseries data with the predicted value
len_data = len(X_Train1)
for i in range (len_data):
    X_Train1[i][18] = y_train_predict[i]
len_data_test = len(X_Test1)
for i in range (len_data_test):
    X_Test1[i][18] = predicted_y_values[i]

In [14]:
mse_for_iter1 = []
train_loss_over_epoch1 = []
val_loss_over_epoch1 = []
predicted_y_values1_unnors = []
models = []

def run_iteration_model_2(no_iter=20, epochs = 10):
    start_time = time.time()
    for iteration in range(no_iter):
        print(f'Iteration: {iteration + 1}')
        K.clear_session()
        model1 = make_model(X_Train1)
        #plot_model(model, show_shapes=True)
        model1.compile(optimizer = 'adam', loss = 'mean_squared_error')
        history = model1.fit(X_Train1, Y_Train1, epochs=epochs,
                       validation_data=(X_Test1, Y_Test1))
        train_loss_over_epoch1.append(history.history['loss'])
        val_loss_over_epoch1.append(history.history['val_loss'])
        predicted_y_values1 = model1.predict(X_Test1)
        predicted_y_values1_unnor = ytrain_min_max_scaler.inverse_transform(predicted_y_values1)
        mse_for_iter1.append(mean_squared_error(predicted_y_values1_unnor, Y_test_actual1))
        predicted_y_values1_unnors.append(predicted_y_values1_unnor)
        models.append(model1)
               
    end_time = time.time()
    print('--------------------------\n')
    print(f'Starting time: {start_time}')
    print(f'Completing time: {end_time}')
    print(f'It took {(start_time - end_time)/60} minutes to train the model for {no_iter} iterations')
    #return predicted unnormalized values for test set from last iteration 
    return predicted_y_values1, predicted_y_values1_unnor, model1

predicted_y_values1, predicted_y_values1_unnor, model1 = run_iteration_model_2(20,10)

Iteration: 1
Epoch 1/10
4638/4638 [==============================] - 136s 29ms/step - loss: 1.2669e-04 - val_loss: 1.0629e-04
Epoch 2/10
4638/4638 [==============================] - 134s 29ms/step - loss: 6.3306e-05 - val_loss: 4.7114e-05
Epoch 3/10
4638/4638 [==============================] - 133s 29ms/step - loss: 4.9841e-05 - val_loss: 4.0540e-05
Epoch 4/10
4638/4638 [==============================] - 134s 29ms/step - loss: 4.1987e-05 - val_loss: 2.9073e-05
Epoch 5/10
4638/4638 [==============================] - 139s 30ms/step - loss: 3.8054e-05 - val_loss: 3.3387e-05
Epoch 6/10
4638/4638 [==============================] - 135s 29ms/step - loss: 3.4034e-05 - val_loss: 2.8730e-05
Epoch 7/10
4638/4638 [==============================] - 136s 29ms/step - loss: 3.1213e-05 - val_loss: 3.3260e-05
Epoch 8/10
4638/4638 [==============================] - 135s 29ms/step - loss: 2.9386e-05 - val_loss: 1.3147e-04
Epoch 9/10
4638/4638 [==============================] - 135s 29ms/step - loss: 2.71

Epoch 3/10
4638/4638 [==============================] - 83s 18ms/step - loss: 4.7748e-05 - val_loss: 3.5486e-05
Epoch 4/10
4638/4638 [==============================] - 82s 18ms/step - loss: 4.0392e-05 - val_loss: 5.7793e-05
Epoch 5/10
4638/4638 [==============================] - 81s 18ms/step - loss: 3.5654e-05 - val_loss: 2.5050e-05
Epoch 6/10
4638/4638 [==============================] - 81s 18ms/step - loss: 3.4612e-05 - val_loss: 4.3667e-05
Epoch 7/10
4638/4638 [==============================] - 81s 18ms/step - loss: 2.7793e-05 - val_loss: 2.4914e-05
Epoch 8/10
4638/4638 [==============================] - 82s 18ms/step - loss: 3.2141e-05 - val_loss: 5.8705e-05
Epoch 9/10
4638/4638 [==============================] - 81s 18ms/step - loss: 2.6097e-05 - val_loss: 2.0244e-05
Epoch 10/10
4638/4638 [==============================] - 82s 18ms/step - loss: 2.5832e-05 - val_loss: 1.8033e-05
Iteration: 9
Epoch 1/10
4638/4638 [==============================] - 87s 19ms/step - loss: 1.2427e-04 -

4638/4638 [==============================] - 82s 18ms/step - loss: 3.5742e-05 - val_loss: 4.2065e-05
Epoch 6/10
4638/4638 [==============================] - 82s 18ms/step - loss: 3.1461e-05 - val_loss: 3.7167e-05
Epoch 7/10
4638/4638 [==============================] - 82s 18ms/step - loss: 3.1221e-05 - val_loss: 4.0814e-05
Epoch 8/10
4638/4638 [==============================] - 82s 18ms/step - loss: 2.7338e-05 - val_loss: 2.4698e-05
Epoch 9/10
4638/4638 [==============================] - 82s 18ms/step - loss: 2.6240e-05 - val_loss: 3.5011e-05
Epoch 10/10
4638/4638 [==============================] - 82s 18ms/step - loss: 2.5828e-05 - val_loss: 2.6814e-05
Iteration: 16
Epoch 1/10
4638/4638 [==============================] - 85s 18ms/step - loss: 1.0115e-04 - val_loss: 5.7940e-05
Epoch 2/10
4638/4638 [==============================] - 84s 18ms/step - loss: 5.4891e-05 - val_loss: 4.1715e-05
Epoch 3/10
4638/4638 [==============================] - 84s 18ms/step - loss: 4.9601e-05 - val_loss:

In [ ]:
model.save("lstm_model2_single_features.hdf5")

In [ ]:
def plot_evaluation():
    #mean square error over iterations
    plt.figure(figsize=(15, 10))
    plt.plot(mse_for_iter1)
    plt.ylabel('mean squared error')
    plt.xlabel('iteration')
    plt.savefig('mse.png', bbox_inches='tight')
    plt.show()
        
    #plot train and validation loss over epochs for each iteration
    fig, axs = plt.subplots(10, 2)
    #axs.set_ylabel('loss')
    #axs.set_xlabel('epoch')
    fig.set_size_inches(18.5, 50)
    for row in range(10):
        for col in range(2):
            index = 2*row + col #index to get loss from training loss and validation loss list
            axs[row, col].plot(train_loss_over_epoch1[index])
            axs[row, col].plot(val_loss_over_epoch1[index])
            axs[row, col].set_title('Iteration(%d)' %(index+1))
            axs[row, col].legend(['train', 'Val'], loc='upper left')            
    plt.savefig('train_val_loss.png', bbox_inches='tight')
    
    #plot predicted value vs actual values from last iteration
    plt.figure(figsize=(15, 10))
    plt.plot(predicted_y_values1_unnor, 'ro', alpha=0.8)
    plt.plot(Y_test_actual1, 'bo', alpha=0.1)
    plt.legend(['Predicted values', 'Actual values'], loc='upper right')
    plt.ylabel('Outflow')
    plt.xlabel('nth datapoint')
    plt.savefig('predicted_values_visuals.png', bbox_inches='tight')
    plt.show()
    
#call plot_evaluation function
plot_evaluation()

In [ ]:
mse_for_iter1

In [ ]:
print(np.mean(mse_for_iter1))

In [ ]:
plot_model(model1, show_shapes=True)